In [4]:
import os
import glob
import numpy as np
import twutils.dvdat as dv
import twutils.pre as twpre
from scipy import constants as C
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets
import IPython.display
import twutils.plot as twplot

# Select buffering mode: True allows time to be a plotting axis, False uses much less memory
# True also allows for better global contrast control
buffered_mode = True

# normalization constants in mks

n1 = 2.8e19*1e6
su = twpre.SimUnits(n1*1e-6)
t1 = su.t1
x1 = su.x1
E1 = su.E1
U1 = C.m_e*C.c*C.c
N1 = n1*x1**3

mpl.rcParams.update({'text.usetex' : False , 'font.size' : 16})

# Starting values

favorite_file = 'Ex.dvdat'
favorite_color = 'jet'
start_path = '/home/gordon/Run/'
file_list = []
color_list = ['viridis','magma','plasma','inferno',
            'Spectral','bwr','seismic','prism','ocean',
            'rainbow','jet','nipy_spectral']
# The following must be in the given form (they are more than labels)
plane_list = ['xytz','zxty','zytx','txyz','tyxz','tzxy']
disp_list = ['Falsecolor-2D','Horizontal-Lineout','Vertical-Lineout']
transform_list = ['abs(x)','Log_e(abs(x))','Log_10(abs(x))','x^2','-x','T[eV]','T[K]','E[V/m]','B[T]']

param_w = ipywidgets.Textarea(value='no data',description='Metrics')
path_w = ipywidgets.Text(value=start_path,description='Path',disabled=False)
file_w = ipywidgets.Dropdown(options=['no files'],value='no files',description='File')
plane_w = ipywidgets.Dropdown(options=plane_list,value='zxty',description='Plane')
disp_w = ipywidgets.Dropdown(options=disp_list,value=disp_list[0],description='Display')
frame_w = ipywidgets.IntSlider(min=0,max=0,step=1,value=0,description='Frame',continuous_update=False)
slice_w = ipywidgets.IntSlider(min=0,max=0,step=1,value=0,description='Slice',continuous_update=False)
hcurs_w = ipywidgets.IntSlider(min=0,max=0,step=1,value=0,description='H-Cursor',continous_update=False)
vcurs_w = ipywidgets.IntSlider(min=0,max=0,step=1,value=0,description='V-Cursor',continous_update=False)
lim1_w = ipywidgets.FloatText(value=-1.0,description='Min')
lim2_w = ipywidgets.FloatText(value=1.0,description='Max')
color_w = ipywidgets.Dropdown(options=color_list,value=favorite_color,description='Colors')
inv_color_w = ipywidgets.Checkbox(value=False,description='Invert Colors')
prop_w = ipywidgets.Checkbox(value=False,description='Proportional')
autoscale_frame_w = ipywidgets.Button(description='Autoscale Frame')
autoscale_all_w = ipywidgets.Button(description='Autoscale All')
transform_select_w = ipywidgets.Dropdown(options=transform_list,value=transform_list[0],layout=ipywidgets.Layout(width='100px'))
transform_w = ipywidgets.Button(description='Transform')
reload_w = ipywidgets.Button(description='Reload')


def setup_files(file_path):
    # Get the file list in the specified path
    os.chdir(file_path)
    file_list = glob.glob('*.dvdat')
    file_list.sort()
    file_w.options = file_list
    if favorite_file in file_list:
        file_w.value = favorite_file
    else:
        file_w.value = file_list[0]
    
def switch_axes_primitive():
    global show_plot
    ax = twplot.get_axis_info(plane_w.value)
    show_plot = False
    frame_w.max = dims[ax[2]]-1
    frame_w.value = 0
    slice_w.max = dims[ax[3]]-1
    slice_w.value = dims[ax[3]]/2
    hcurs_w.max = dims[ax[0]]-1
    hcurs_w.value = dims[ax[0]]/2
    vcurs_w.max = dims[ax[1]]-1
    vcurs_w.value = dims[ax[1]]/2
    show_plot = True

def switch_axes(plane_spec):
    switch_axes_primitive()

def setup_plots(file_to_plot):
    global plotter
    global phys_dims
    global dims
    global show_plot
    theFile = path_w.value + file_to_plot
    plotter = twplot.plotter(theFile,buffered=buffered_mode)
    dims = plotter.dims4()
    phys_dims = plotter.phys_dims4()
    param_w.value = '(t,x,y,z) = '+str(dims)+'\n'
    switch_axes_primitive()
    auto_scale_all(autoscale_all_w)

def auto_scale_frame(the_button):
    ax = twplot.get_axis_info(plane_w.value)
    # Extract the frame slice
    frame_slice = np.take(plotter.get(),[frame_w.value],axis=ax[2])
    # Extract the "slice slice"
    data_slice = np.take(frame_slice,[slice_w.value],axis=ax[3])
    lim1_w.value = np.min(data_slice)
    lim2_w.value = np.max(data_slice)
    
def auto_scale_all(the_button):
    lim1_w.value = np.min(plotter.get())
    lim2_w.value = np.max(plotter.get())

def reload_data(the_button):
    setup_plots(file_w.value)
    
def transform_data(the_button):
    smallest_num = 1e-25
    if transform_select_w.value=='Log_e(abs(x))':
        plotter.set(np.log(smallest_num+np.abs(plotter.get())))
    if transform_select_w.value=='Log_10(abs(x))':
        plotter.set(np.log10(smallest_num+np.abs(plotter.get())))
    if transform_select_w.value=='x^2':
        plotter.set(plotter.get()**2)
    if transform_select_w.value=='-x':
        plotter.scale(-1.0)
    if transform_select_w.value=='T[eV]':
        plotter.scale(U1/C.e)
    if transform_select_w.value=='T[K]':
        plotter.scale(U1/C.k)
    if transform_select_w.value=='E[V/m]':
        plotter.scale(E1)
    if transform_select_w.value=='B[T]':
        plotter.scale(E1/C.c)
    if transform_select_w.value=='abs(x)':
        plotter.set(np.abs(plotter.get()))
    auto_scale_all(autoscale_all_w)
        
def disp_frame(frame_to_plot,slice_to_plot,hcurs,vcurs,color_map,inv_color,prop,lim2,disp):
    if show_plot:
        if inv_color:
            color_map = color_map + '_r'
        if prop:
            my_aspect = 'equal'
        else:
            my_aspect = 'auto'
        slicing = plane_w.value
        fig = plt.figure(1,figsize=(14,9),dpi=140)
        if disp=='Falsecolor-2D':
            data_slice,plot_dict = plotter.falsecolor2d(slicing,(frame_to_plot,slice_to_plot))
            ax = fig.add_subplot(111)
            myPlot = plt.imshow(data_slice,
                                vmin=lim1_w.value,
                                vmax=lim2_w.value,
                                origin='lower',
                                aspect=my_aspect,
                                extent=plot_dict['extent'],
                                cmap=color_map)
            hc = plotter.plot_coord(plot_dict,0,hcurs_w.value)
            vc = plotter.plot_coord(plot_dict,1,vcurs_w.value) 
            plt.plot([hc],[vc],'ro')
            #ax.text(hc,vc,'+')
            plt.colorbar()
            plt.xlabel(plot_dict['xlabel'],fontsize=18)
            plt.ylabel(plot_dict['ylabel'],fontsize=18)
        if disp=='Horizontal-Lineout':
            abcissa,ordinate,plot_dict = plotter.lineout(slicing,(vcurs,frame_to_plot,slice_to_plot))
            plt.plot(abcissa,ordinate)
            plt.xlabel(plot_dict['xlabel'],fontsize=18)
            plt.ylabel(plot_dict['ylabel'],fontsize=18)
        if disp=='Vertical-Lineout':
            vslicing = slicing[1]+slicing[0]+slicing[2]+slicing[3]
            abcissa,ordinate,plot_dict = plotter.lineout(vslicing,(hcurs,frame_to_plot,slice_to_plot))
            plt.plot(abcissa,ordinate)
            plt.xlabel(plot_dict['xlabel'],fontsize=18)
            plt.ylabel(plot_dict['ylabel'],fontsize=18)
        plt.show()

# Set up interactive elements

setup_iw = ipywidgets.interactive(setup_plots,file_to_plot = file_w)

update_iw = ipywidgets.interactive(disp_frame,
                    frame_to_plot=frame_w,
                    slice_to_plot=slice_w,
                    hcurs=hcurs_w,
                    vcurs=vcurs_w,
                    color_map=color_w,
                    inv_color=inv_color_w,
                    prop=prop_w,
                    lim2=lim2_w,
                    disp=disp_w)

switch_iw = ipywidgets.interactive(switch_axes,plane_spec=plane_w)
autoscale_frame_w.on_click(auto_scale_frame)
autoscale_all_w.on_click(auto_scale_all)
transform_w.on_click(transform_data)
reload_w.on_click(reload_data)

# Set up layouts

autoscale_box = ipywidgets.HBox([autoscale_frame_w,autoscale_all_w])
transform_box = ipywidgets.HBox([transform_select_w,transform_w])
left_box = ipywidgets.VBox([param_w,path_w,file_w,lim1_w,lim2_w,color_w,inv_color_w,
                            prop_w,frame_w,slice_w,hcurs_w,vcurs_w,plane_w,disp_w,autoscale_box,transform_box,reload_w],
                            layout=ipywidgets.Layout(justify_content='flex-start',width='800px',flex='1 3 auto'))

update_iw.children[-1].layout.flex = '4 2 auto'
main_view = ipywidgets.HBox([left_box, update_iw.children[-1]],
                            layout=ipywidgets.Layout(display='inline-flex',
                                                     align_items='stretch',
                                                     align_content='stretch',
                                                     justify_content='flex-start'))

IPython.display.display(main_view)

# Initialize plots

setup_files(path_w.value)